## 3D Case

In [1]:
#%% Libraries import
# import HiDeNN library
import sys  
# sys.path.append("../neurom/")
from neurom.HiDeNN_PDE import MeshNN, NeuROM, MeshNN_2D, MeshNN_1D, MeshNN_3D
# Import pre-processing functions
import neurom.src.Pre_processing as pre
# Import torch librairies
import torch
import torch.nn as nn
# Import mechanical functions
from neurom.src.PDE_Library import Strain, Stress,VonMises_plain_strain
# Import Training funcitons
from neurom.src.Training import Training_2_3D_FEM
#Import post processing libraries
import neurom.Post.Plots as Pplot
import time
import os
import torch._dynamo as dynamo
mps_device = torch.device("mps")
from importlib import reload  # Python 3.4+
import tomllib
import numpy as numpy
import argparse
import meshio

import matplotlib.pyplot as plt



* WARNING: could not load tikzplotlib


Load configuration file for 3D case

In [2]:
Default_config_file = 'Configurations/config_3D_Cube_w_hole.toml'

with open(Default_config_file, mode="rb") as f:
    config = tomllib.load(f)

# Load evaluation data

- sampling points 
- reference values of displacement and stress

In [3]:
eval_coord_file = "GroundTruth/Cube_eval_coordinates_0.05.npy"
num_displ_file = "GroundTruth/Cube_hole_order_1_0.05_displacement.npy"
num_stress_file = "GroundTruth/Cube_hole_order_1_0.05_stress.npy"

eval_coord =  torch.tensor(numpy.load(eval_coord_file), dtype=torch.float64, requires_grad=True)
n_samples =   eval_coord.shape[0]

eval_coord = eval_coord[0:n_samples,:]
num_displ = torch.tensor(numpy.load(num_displ_file))[0:n_samples,:]
num_stress = torch.tensor(numpy.load(num_stress_file))[0:n_samples,:]
num_stress = torch.index_select(num_stress, 1, torch.LongTensor([0,4,8,1,5,2]))

num_times = [0.062, 0.109, 0.682, 5.809, 101.365]


# Ordering of stress componennts: 

# xx xy xz      0 1 2
# yx yy yz      3 4 5
# zx zy zz      6 7 8

# nn stress 3D: XX, YY, ZZ, XY, YZ, XZ  ---> 0 : xx, 4:yy, 8: zz, 1: xy, 5: yz, 2: xz


# Table 2 - Fixed mesh training

- Initialize model based on mesh resolution
- Load trained weights
- Evaluate model on sampling points + compute stress

In [4]:
experiments_list = ["Experiment_1", "Experiment_2", "Experiment_3", "Experiment_4"]
element_sizes_list = [[1, 0.5, 0.25, 0.125, 0.0625],[1, 0.5, 0.25, 0.125, 0.0625],[1, 0.5, 0.25], [1, 0.5, 0.25]]

optimizers = ["lbfgs", "adam"]

error_stress = []
error_displ = []
training_time = []


for exp in range(len(experiments_list)):

    element_size = element_sizes_list[exp]
    experiment = experiments_list[exp]

    print("Experiment: ", experiment)

    error_stress.append(numpy.zeros((len(optimizers), len(element_size))))
    error_displ.append(numpy.zeros((len(optimizers), len(element_size))))
    training_time.append(numpy.zeros((len(optimizers), len(element_size))))

    for e in range(len(element_size)):
        print("     Resolution : ", element_size[e])

        for o_id in range(len(optimizers)):
            # elem_IDs = []
            o = optimizers[o_id]
            print("     Opimizer: ", o)

            time = torch.tensor(numpy.load('Trained_models/'+experiment+'/'+o+'_3D_Cube_'+str(element_size[e])+'_time.npy'))
            trained_dict = torch.load('Trained_models/'+experiment+'/'+o+'_3D_Cube_'+str(element_size[e]), weights_only=False)
            config["interpolation"]["MaxElemSize2D"] = trained_dict["h_max"].item()

            if config["interpolation"]["dimension"] == 1:
                Mat = pre.Material(         flag_lame = True,                               # If True should input lmbda and mu instead of E and nu
                                            coef1     = config["material"]["E"],            # Young Modulus
                                            coef2     = config["geometry"]["A"]             # Section area of the 1D bar
                                    )
            elif config["interpolation"]["dimension"] in [2, 3]:
                try:
                    Mat = pre.Material(     flag_lame = False,                              # If True should input lmbda and mu instead of E and nu
                                            coef1     = config["material"]["E"],            # Young Modulus
                                            coef2     = config["material"]["nu"]            # Poisson's ratio
                                    )
                except:
                    Mat = pre.Material(     flag_lame = True,                               # If True should input lmbda and mu instead of E and nu
                                            coef1     = config["material"]["lmbda"],        # First Lame's coef
                                            coef2     = config["material"]["mu"]            # Second Lame's coef
                                    )

            MaxElemSize = pre.ElementSize(
                                            dimension     = config["interpolation"]["dimension"],
                                            L             = config["geometry"]["L"],
                                            order         = config["interpolation"]["order"],
                                            np            = config["interpolation"]["np"],
                                            MaxElemSize2D = config["interpolation"]["MaxElemSize2D"]
                                        )
            Excluded = []
            Mesh_object = pre.Mesh( 
                                            config["geometry"]["Name"],                 # Create the mesh object
                                            MaxElemSize, 
                                            config["interpolation"]["order"], 
                                            config["interpolation"]["dimension"]
                                    )
            Mesh_object.AddBorders(config["Borders"]["Borders"])
            Mesh_object.AddBCs(                                                         # Include Boundary physical domains infos (BCs+volume)
                                            config["geometry"]["Volume_element"],
                                            Excluded,
                                            config["DirichletDictionryList"]
                                )                   

            Mesh_object.MeshGeo()                                                       # Mesh the .geo file if .msh does not exist
            Mesh_object.ReadMesh()                                                      # Parse the .msh file
            Mesh_object.ExportMeshVtk()

            if int(Mesh_object.dim) != int(Mesh_object.dimension):
                raise ValueError("The dimension of the provided geometry does not match the job dimension")

            Model_FEM = MeshNN_3D(Mesh_object, n_components = 3)
            Model_FEM.load_state_dict(trained_dict)
            Model_FEM.eval()


            coordinates_all = torch.ones_like(Model_FEM.coordinates_all)
            coordinates_all[Model_FEM.coord_free] = Model_FEM.coordinates['free']
            coordinates_all[~Model_FEM.coord_free] = Model_FEM.coordinates['imposed']
            Nodes = torch.hstack([torch.linspace(1,coordinates_all.shape[0],coordinates_all.shape[0], dtype = coordinates_all.dtype, device = coordinates_all.device)[:,None],
                                    coordinates_all])
            Model_FEM.mesh.Nodes = Nodes.detach().cpu().numpy()
            Model_FEM.mesh.ExportMeshVtk(flag_update = True)

            elem_IDs = torch.tensor(Model_FEM.mesh.GetCellIds(eval_coord),dtype=torch.int)

            u = Model_FEM(eval_coord, elem_IDs)
            eps =  Strain(u,eval_coord)
            sigma =  torch.stack(Stress(eps, Mat.lmbda, Mat.mu),dim=1)
            u = torch.transpose(u,0,1)

            error_displ[exp][o_id,e] = (torch.linalg.vector_norm(num_displ - u)/torch.linalg.vector_norm(num_displ)).item()
            error_stress[exp][o_id,e] = (torch.linalg.vector_norm(num_stress - sigma)/torch.linalg.vector_norm(num_stress)).item()
            training_time[exp][o_id,e] = time




Experiment:  Experiment_1
     Resolution :  1
     Opimizer:  lbfgs
 
 
   _   _            ____   ___  __  __ 
  | \ | | ___ _   _|  _ \ / _ \|  \/  |
  |  \| |/ _ \ | | | |_) | | | | |\/| |
  | |\  |  __/ |_| |  _ <| |_| | |  | |
  |_| \_|\___|\__,_|_| \_\ ___/|_|  |_|

                  2024.09.18
*************** Mesh Geometry  ****************

Info    : Running '/opt/homebrew/Caskroom/miniconda/base/envs/neurom_1/bin/gmsh Geometries/Cube_hole.geo -3 -order 1 -o Geometries/Cube_hole_order_1_1.msh -clmax 1' [Gmsh 4.13.1, 1 node, max. 1 thread]
Info    : Started on Thu Apr 10 10:28:54 2025
Info    : Reading 'Geometries/Cube_hole.geo'...
Info    : Done reading 'Geometries/Cube_hole.geo'                                                                         
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 10%] Meshing curve 3 (Line)
Info    : [ 20%] Meshing curve 4 (Line)
Info    : [ 20%] Meshing curve 5 (Circle)
Inf

Warning: DOLFIN XML is a legacy format. Consider using XDMF instead.

Warning: DOLFIN XML is a legacy format. Consider using XDMF instead.

     Opimizer:  adam
 
 
   _   _            ____   ___  __  __ 
  | \ | | ___ _   _|  _ \ / _ \|  \/  |
  |  \| |/ _ \ | | | |_) | | | | |\/| |
  | |\  |  __/ |_| |  _ <| |_| | |  | |
  |_| \_|\___|\__,_|_| \_\ ___/|_|  |_|

                  2024.09.18

************ MESH READING COMPLETE ************

 * Dimension of the problem: 3D
 * Elements type:            4-node tetrahedron
 * Number of Dofs:           2403



Warning: DOLFIN XML is a legacy format. Consider using XDMF instead.

...

Warning: DOLFIN XML is a legacy format. Consider using XDMF instead.

Warning: DOLFIN XML is a legacy format. Consider using XDMF instead.

     Opimizer:  adam
 
 
   _   _            ____   ___  __  __ 
  | \ | | ___ _   _|  _ \ / _ \|  \/  |
  |  \| |/ _ \ | | | |_) | | | | |\/| |
  | |\  |  __/ |_| |  _ <| |_| | |  | |
  |_| \_|\___|\__,_|_| \_\ ___/|_|  |_|

                  2024.09.18

************ MESH READING COMPLETE ************

 * Dimension of the problem: 3D
 * Elements type:            4-node tetrahedron
 * Number of Dofs:           52137



Warning: DOLFIN XML is a legacy format. Consider using XDMF instead.

Warning: DOLFIN XML is a legacy format. Consider using XDMF instead.

# Numerical solution vs Reference solution

- load FEM solution and evaluate error

In [8]:
element_size = [1, 0.5, 0.25, 0.125, 0.062]
num_error_stress = numpy.zeros(len(element_size))
num_eroor_displ = numpy.zeros(len(element_size))
max_stress = numpy.zeros(len(element_size))

for e in range(len(element_size)):
    print("Resolution : ", element_size[e])

    displ_file = "../../3D_example/num/Cube_hole_order_1_"+str(element_size[e])+"_displacement.npy"
    stress_file = "../../3D_example/num/Cube_hole_order_1_"+str(element_size[e])+"_stress.npy"

    displ = torch.tensor(numpy.load(displ_file))[0:n_samples,:]
    stress = torch.tensor(numpy.load(stress_file))[0:n_samples,:]
    stress = torch.index_select(stress, 1, torch.LongTensor([0,4,8,1,5,2]))

    num_eroor_displ[e] = (torch.linalg.vector_norm(num_displ - displ)/torch.linalg.vector_norm(num_displ)).item()
    num_error_stress[e] = (torch.linalg.vector_norm(num_stress - stress)/torch.linalg.vector_norm(num_stress)).item()






Resolution :  1
Resolution :  0.5
Resolution :  0.25
Resolution :  0.125
Resolution :  0.062


# Table 2: Fixed mesh

In [9]:

for e in range(len(element_sizes_list[0])):
    if e==0:
        print("Level: ", e+1)
        print("     FEM : ", numpy.round(num_eroor_displ[e], decimals=4), numpy.round(num_error_stress[e], decimals=4), num_times[e] )
        print("     L-BFGS: ",        numpy.round(error_displ[1][0,e],decimals=4), numpy.round(error_stress[1][0,e],decimals=4), numpy.round(training_time[1][0,e],decimals=3))
        print("     Adam + L-BFGS: ", numpy.round(error_displ[1][1,e],decimals=4), numpy.round(error_stress[1][1,e],decimals=4), numpy.round(training_time[1][1,e],decimals=3))
        print()
    else:
        print("Level: ", e+1)
        print("     FEM : ", numpy.round(num_eroor_displ[e], decimals=4), numpy.round(num_error_stress[e], decimals=4), num_times[e] )
        print("     single-level, L-BFGS: ",        numpy.round(error_displ[1][0,e],decimals=4), numpy.round(error_stress[1][0,e],decimals=4), numpy.round(training_time[1][0,e],decimals=3))
        print("     single-level, Adam + L-BFGS: ", numpy.round(error_displ[1][1,e],decimals=4), numpy.round(error_stress[1][1,e],decimals=4), numpy.round(training_time[1][1,e],decimals=3))
        print("     multi-level, L-BFGS: ",         numpy.round(error_displ[0][0,e],decimals=4), numpy.round(error_stress[0][0,e],decimals=4), numpy.round(training_time[0][0,e],decimals=3))
        print("     multi-level, Adam + L-BFGS: ",  numpy.round(error_displ[0][1,e],decimals=4), numpy.round(error_stress[0][1,e],decimals=4), numpy.round(training_time[0][1,e],decimals=3))
        print()
  








Level:  1
     FEM :  0.1062 0.4194 0.062
     L-BFGS:  0.1062 0.4194 2.164
     Adam + L-BFGS:  0.1061 0.4194 8.799

Level:  2
     FEM :  0.0664 0.3217 0.109
     single-level, L-BFGS:  0.0664 0.3218 6.468
     single-level, Adam + L-BFGS:  0.0664 0.3218 22.823
     multi-level, L-BFGS:  0.0664 0.3218 9.472
     multi-level, Adam + L-BFGS:  0.0664 0.3218 28.292

Level:  3
     FEM :  0.0227 0.1889 0.682
     single-level, L-BFGS:  0.0227 0.1889 46.714
     single-level, Adam + L-BFGS:  0.0227 0.1889 108.696
     multi-level, L-BFGS:  0.0227 0.1889 58.197
     multi-level, Adam + L-BFGS:  0.0227 0.1889 102.74

Level:  4
     FEM :  0.0062 0.11 5.809
     single-level, L-BFGS:  0.0062 0.11 515.266
     single-level, Adam + L-BFGS:  0.0064 0.11 739.592
     multi-level, L-BFGS:  0.0063 0.11 171.457
     multi-level, Adam + L-BFGS:  0.0063 0.11 347.136

Level:  5
     FEM :  0.0007 0.0667 101.365
     single-level, L-BFGS:  0.0043 0.0667 6022.36
     single-level, Adam + L-BFGS:  0.0012 

# Table 3: r-adaptive mesh

In [10]:
for e in range(len(element_sizes_list[0])):
    if e==0:
        print("Level: ", e+1)
        print("     FEM : ", numpy.round(num_eroor_displ[e], decimals=4), numpy.round(num_error_stress[e], decimals=4), num_times[e] )
        print("     L-BFGS: ",        numpy.round(error_displ[2][0,e],decimals=4), numpy.round(error_stress[2][0,e],decimals=4), numpy.round(training_time[2][0,e],decimals=3))
        print("     Adam + L-BFGS: ", numpy.round(error_displ[2][1,e],decimals=4), numpy.round(error_stress[2][1,e],decimals=4), numpy.round(training_time[2][1,e],decimals=3))
        print()
    elif e>0 and e<3:
        print("Level: ", e+1)
        print("     FEM : ", numpy.round(num_eroor_displ[e], decimals=4), numpy.round(num_error_stress[e], decimals=4), num_times[e] )
        print("     single-level, L-BFGS: ",        numpy.round(error_displ[2][0,e],decimals=4), numpy.round(error_stress[2][0,e],decimals=4), numpy.round(training_time[2][0,e],decimals=3))
        print("     single-level, Adam + L-BFGS: ", numpy.round(error_displ[3][1,e],decimals=4), numpy.round(error_stress[3][1,e],decimals=4), numpy.round(training_time[3][1,e],decimals=3))
        print("     multi-level, L-BFGS: ",         numpy.round(error_displ[3][0,e],decimals=4), numpy.round(error_stress[3][0,e],decimals=4), numpy.round(training_time[3][0,e],decimals=3))
        print("     multi-level, Adam + L-BFGS: ",  numpy.round(error_displ[3][1,e],decimals=4), numpy.round(error_stress[3][1,e],decimals=4), numpy.round(training_time[3][1,e],decimals=3))
        print()



Level:  1
     FEM :  0.1062 0.4194 0.062
     L-BFGS:  0.1062 0.4191 3.774
     Adam + L-BFGS:  0.1062 0.4191 10.946

Level:  2
     FEM :  0.0664 0.3217 0.109
     single-level, L-BFGS:  0.0641 0.321 80.557
     single-level, Adam + L-BFGS:  0.0637 0.3111 46.237
     multi-level, L-BFGS:  0.0627 0.3115 26.873
     multi-level, Adam + L-BFGS:  0.0637 0.3111 46.237

Level:  3
     FEM :  0.0227 0.1889 0.682
     single-level, L-BFGS:  0.1386 0.237 1086.93
     single-level, Adam + L-BFGS:  0.0215 0.1664 106.09
     multi-level, L-BFGS:  0.0206 0.1673 144.599
     multi-level, Adam + L-BFGS:  0.0215 0.1664 106.09

